In [3]:
import csv
import os
import requests
import json
import time
import io
import datetime;
import pandas as pd
from tqdm import tqdm
import magic

os.chdir(r"D:\Users\snangare\Sandesh\API")
file_object = open('logs/source_export/source_documents_all_versions.txt', 'a')


def authenticate():
    
    endpoint = 'https://amgensbx-etmf.veevavault.com/api/v20.3/auth' 
    parameters = {
        'username':'snangare@amgensbx.com',
        'password':'A1ph@12345'
    }
    
    try: 
        response = requests.post(endpoint, data=parameters).json()
        if response['responseStatus'] != 'SUCCESS':
            raise Exception()
        else:
            return response['sessionId']
    except:
        error_message = response['errorType'] + ', Reason: ' + response['responseMessage']
        print (error_message)
        
def fetch_documents(session_id,endpoint,doc_name,major_version,minor_version,doc_id,document_name):
    
    headers = {
        'Authorization': session_id
    }
    
    response = requests.get(endpoint, headers=headers)
    
    file= doc_name+'_'+document_name+'_'+ doc_id +'_'+ major_version+'_'+ minor_version
    
    try :
        if response.json():
            
            response = response.json()
            
            if response['responseStatus'] != 'SUCCESS':
                file_object.write(file+" was not available for export at "+str(datetime.datetime.now())+" IST")
                file_object.write('\n')
            else:
                file_object.write("Something went wrong with "+file[:file.index('viewable')-1]+".pdf")
                file_object.write('\n')
    except:
        with open(file, 'wb') as f:
            f.write(response.content)
            
            
        if 'PDF' in str(magic.from_file(file)):
            os.rename(file, file+'.pdf')
            file = file + '.pdf'            
        elif 'with very long lines' in str(magic.from_file(file)) or 'ASCII text' in str(magic.from_file(file)):
            os.rename(file, file+'.csv')
            file = file + '.csv'
        elif 'Microsoft Word 2007+' in str(magic.from_file(file)):
            os.rename(file, file+'.docx')
            file = file + '.docx'
        elif 'Microsoft Excel 2007+' in str(magic.from_file(file)):
            os.rename(file, file+'.xlsx')   
            file = file + '.xlsx'
        elif 'Microsoft Outlook Message' in str(magic.from_file(file)):
            os.rename(file, file+'.msg')   
            file = file + '.msg'
        elif 'Microsoft Office Word' in str(magic.from_file(file)):
            os.rename(file, file+'.doc')   
            file = file + '.doc'   

        file_object.write(file+" exported successfully at "+str(datetime.datetime.now())+" IST")
        file_object.write('\n') 
        
def execute():
    session_id = authenticate()
    
    df = pd.read_csv('Document Export.csv')
    os.chdir("exports/source_export")
    
    try:
        
        for ind in tqdm(df.index): 
            doc_id = str(df['Document ID'][ind])
            document_name = str(df['Document Name'][ind])
            doc_name = str(df['Document Number'][ind])
            major_version = str(df['Major Version Number'][ind])
            minor_version = str(df['Minor Version Number'][ind])

            URI = 'https://amgensbx-etmf.veevavault.com/api/v20.3/'
            action = 'objects/documents/'+str(doc_id)+'/versions/'+major_version+'/'+minor_version+'/file            endpoint = URI + action

            fetch_documents(session_id,endpoint,doc_name,major_version,minor_version,doc_id,document_name)
    finally :
        
        file_object.close()
    
execute()  

SyntaxError: EOL while scanning string literal (<ipython-input-3-663cead1cd84>, line 98)